In [21]:
import moxing as mox

for fileid in range(17):
    fileid = str(fileid).zfill(2)
    mox.file.copy(f's3://bucket-1784/Qiaojunbo/Dataset/LS/shard-{fileid}.tar.gz', f'/cache/xiehaizhen/shard-{fileid}.tar.gz')


In [1]:
import os
from os.path import join
import cv2 as cv
import imgaug.augmenters as iaa
from PIL import Image
import numpy as np
import tqdm
import shutil
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models

from _curriculum_clustering import CurriculumClustering

model = models.vgg19(pretrained=True)
model.to('cuda:0')

# checkpoint = torch.load('./weights/resnet_classify_map_epoch200.pth')
# model.load_state_dict(checkpoint)

# activation = []
# def get_activation_hook(model, input, output):
#     activation.append(output.clone().detach())
# hook = model.avgpool.register_forward_hook(get_activation_hook)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [58]:
for kk in range(7, 86):
    print('文件夹', kk)

    SIZE_PADDING = 512
    SIZE_CROPPING = 448

    X = []
    names = []

    img_paths = '/home/ma-user/work/xiehaizhen/NTIRE23-RTSR-master/src/result/inpx2'
    img_names = os.listdir(img_paths)
    img_names.sort()  # 排序
    img_names = [os.path.join(img_paths, name) for name in img_names]

    LSDIR = str(kk).zfill(4) + '000'
    img_paths2 = '/cache/xiehaizhen/' + LSDIR
    img_names2 = os.listdir(img_paths2)
    img_names2.sort()  # 排序
    img_names2 = [os.path.join(img_paths2, name) for name in img_names2]

    img_names = img_names + img_names2

    for img_name in tqdm.tqdm(img_names):
        if img_name.endswith('png') or img_name.endswith('PNG') \
                or img_name.endswith('jpeg') or img_name.endswith('JPEG') \
                or img_name.endswith('jpg') or img_name.endswith('webp'):

            # try:
            image = Image.open(img_name)
            if image.mode != 'RGB':
                image = image.convert("RGB")
            image = np.array(image)
            h, w = image.shape[:2]
            names.append(img_name)
            # except Exception as exe:
            #     print(img_name)
            #     continue

            aug = iaa.Sequential([
                    iaa.Resize({"height": SIZE_PADDING, "width": "keep-aspect-ratio"}) if h > w else
                    iaa.Resize({"height": "keep-aspect-ratio", "width": SIZE_PADDING}),
                    iaa.PadToFixedSize(width=SIZE_PADDING, height=SIZE_PADDING, position="center"),
                    iaa.CenterCropToFixedSize(height=SIZE_CROPPING, width=SIZE_CROPPING)
                ])
            image_aug = aug.augment_image(image)

            transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            image_trans = transform(image_aug)
            image_trans = image_trans.cuda().unsqueeze(0)

            # 获取中间结果的方法一
            # rst = nn.Sequential(*list(model.children())[:-1])(image_trans)
            # print('rst1:', rst)

            # model(image_trans)
            # hook_layer = activation[-1]
            # hook_layer = hook_layer.view(hook_layer.shape[0], -1).squeeze()
            # hook_layer = hook_layer.data.cpu().numpy()

            a = model.features(image_trans)  # 使用feature获取特征层（卷积层）的特征；输出特征维度为【1，512，4，4】
            b = model.avgpool(a)  # 使用vgg定义的池化操作；输出特征维度为【1，512，7，7】
            c = torch.flatten(b, 1)  # 将特征变成一维度；输出特征维度为【1，25088】
            d = model.classifier[:2](c)  # 使用分类层的的第一层，当然可以选择数；输出特征维度为【1，4096】
            X.append(d.squeeze().data.cpu().numpy())

    # hook.remove()


    X = np.array(X)
    Y = [0] * X.shape[0]
    print(X.shape)

    n_subsets = 12
    dim_reduce = 256
    calc_auxiliary = False
    cc = CurriculumClustering(n_subsets=n_subsets, dim_reduce=dim_reduce, calc_auxiliary=calc_auxiliary)
    result, auxiliary_info = cc.fit_predict(X, Y)

    # print(result)

    result = np.array(result)
    names = np.array(names)

    art = [124,127,130,131,132,133,134,138,139,140]
    game = [100,101,102,103,104,105,106,107,111,112]

    arr = [[] for _ in range(n_subsets)]
    art_arr = [[] for _ in range(n_subsets)]
    game_arr = [[] for _ in range(n_subsets)]

    for i, img_name in zip(result, names):
        if img_paths in img_name and int(os.path.basename(img_name).split('.')[0]) > 100:
            arr[i].append(os.path.basename(img_name))
        if img_paths in img_name and int(os.path.basename(img_name).split('.')[0]) in game:
            game_arr[i].append(os.path.basename(img_name))
        if img_paths in img_name and int(os.path.basename(img_name).split('.')[0]) in art:
            art_arr[i].append(os.path.basename(img_name))

    print('每个类总计：', [sum(result == i) for i in range(n_subsets)], '\n')
    for i in range(n_subsets):
        print(i, arr[i])
    print('\n', 'game')
    for i in range(n_subsets):
        print(i, art_arr[i])
    print('\n', 'art')
    for i in range(n_subsets):
        print(i, game_arr[i])

    select_set = []
    for i in range(n_subsets):
        length = len(art_arr[i]) + len(game_arr[i])
        print(i, length)
        if length >= 3:
            select_set.append(i)

    # select_set = [0,1,2]
    for i in select_set:
        os.system('rm -rf /cache/xiehaizhen/CurriculumClustering/{}/{}'.format(LSDIR, str(i)))
        os.makedirs('/cache/xiehaizhen/CurriculumClustering/{}/{}'.format(LSDIR, str(i)))
        for img_name in names[result == i]:
            if img_paths not in img_name: # 把val中的图片去除
                shutil.copy(img_name, join('/cache/xiehaizhen/CurriculumClustering/{}/{}'.format(LSDIR, str(i)), os.path.basename(img_name)))

文件夹 7


100%|██████████| 1150/1150 [01:14<00:00, 15.35it/s]


(1150, 4096)
每个类总计： [266, 211, 139, 120, 70, 58, 29, 32, 27, 23, 42, 133] 

0 ['101.jpg', '102.jpg', '103.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg', '111.jpg', '114.jpg', '115.jpg', '116.jpg', '117.jpg', '119.jpg', '120.jpg', '121.jpg', '125.jpg', '126.jpg', '128.jpg', '131.jpg', '133.jpg', '134.jpg', '136.jpg', '138.jpg', '139.jpg', '141.jpg', '143.jpg', '146.jpg', '148.jpg', '149.jpg']
1 ['108.jpg', '110.jpg', '112.jpg', '124.jpg', '130.jpg', '135.jpg', '137.jpg', '147.jpg']
2 ['118.jpg', '123.jpg', '127.jpg', '132.jpg', '140.jpg', '144.jpg']
3 ['113.jpg', '122.jpg', '129.jpg']
4 []
5 []
6 []
7 ['109.jpg']
8 []
9 []
10 ['142.jpg']
11 ['145.jpg']

 game
0 ['131.jpg', '133.jpg', '134.jpg', '138.jpg', '139.jpg']
1 ['124.jpg', '130.jpg']
2 ['127.jpg', '132.jpg', '140.jpg']
3 []
4 []
5 []
6 []
7 []
8 []
9 []
10 []
11 []

 art
0 ['100.jpg', '101.jpg', '102.jpg', '103.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg', '111.jpg']
1 ['112.jpg']
2 []
3 []
4 []
5 []
6 []
7 []
8 []
9 []
10

100%|██████████| 1150/1150 [01:09<00:00, 16.44it/s]


(1150, 4096)
每个类总计： [253, 201, 139, 111, 73, 51, 42, 32, 32, 22, 31, 163] 

0 ['101.jpg', '102.jpg', '103.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg', '111.jpg', '114.jpg', '115.jpg', '116.jpg', '117.jpg', '119.jpg', '120.jpg', '121.jpg', '126.jpg', '128.jpg', '131.jpg', '133.jpg', '134.jpg', '136.jpg', '138.jpg', '139.jpg', '141.jpg', '143.jpg', '146.jpg', '148.jpg']
1 ['108.jpg', '110.jpg', '112.jpg', '123.jpg', '124.jpg', '125.jpg', '130.jpg', '135.jpg', '137.jpg', '147.jpg', '149.jpg']
2 ['118.jpg', '127.jpg', '132.jpg', '140.jpg', '144.jpg']
3 ['113.jpg', '122.jpg', '129.jpg']
4 []
5 ['109.jpg']
6 []
7 []
8 []
9 ['142.jpg']
10 []
11 ['145.jpg']

 game
0 ['131.jpg', '133.jpg', '134.jpg', '138.jpg', '139.jpg']
1 ['124.jpg', '130.jpg']
2 ['127.jpg', '132.jpg', '140.jpg']
3 []
4 []
5 []
6 []
7 []
8 []
9 []
10 []
11 []

 art
0 ['100.jpg', '101.jpg', '102.jpg', '103.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg', '111.jpg']
1 ['112.jpg']
2 []
3 []
4 []
5 []
6 []
7 []
8 []
9 []
10

...

(1150, 4096)
每个类总计： [171, 231, 182, 122, 70, 72, 46, 32, 27, 28, 30, 139] 

0 ['101.jpg', '102.jpg', '105.jpg', '106.jpg', '107.jpg', '114.jpg', '115.jpg', '116.jpg', '117.jpg', '119.jpg', '121.jpg', '126.jpg', '136.jpg', '139.jpg', '143.jpg', '146.jpg', '148.jpg']
1 ['103.jpg', '104.jpg', '111.jpg', '120.jpg', '125.jpg', '128.jpg', '131.jpg', '133.jpg', '134.jpg', '138.jpg', '141.jpg']
2 ['108.jpg', '110.jpg', '112.jpg', '124.jpg', '135.jpg', '137.jpg', '147.jpg', '149.jpg']
3 ['118.jpg', '123.jpg', '130.jpg']
4 ['144.jpg']
5 ['127.jpg', '132.jpg', '140.jpg']
6 ['113.jpg']
7 ['122.jpg']
8 []
9 ['129.jpg']
10 []
11 ['109.jpg', '142.jpg', '145.jpg']

 game
0 ['139.jpg']
1 ['131.jpg', '133.jpg', '134.jpg', '138.jpg']
2 ['124.jpg']
3 ['130.jpg']
4 []
5 ['127.jpg', '132.jpg', '140.jpg']
6 []
7 []
8 []
9 []
10 []
11 []

 art
0 ['100.jpg', '101.jpg', '102.jpg', '105.jpg', '106.jpg', '107.jpg']
1 ['103.jpg', '104.jpg', '111.jpg']
2 ['112.jpg']
3 []
4 []
5 []
6 []
7 []
8 []
9 []
10 []
11 []
0 

100%|██████████| 1150/1150 [01:42<00:00, 11.21it/s]


(1150, 4096)
每个类总计： [211, 190, 156, 117, 80, 56, 49, 44, 42, 23, 30, 152] 

0 ['101.jpg', '102.jpg', '105.jpg', '106.jpg', '107.jpg', '114.jpg', '115.jpg', '116.jpg', '117.jpg', '119.jpg', '121.jpg', '126.jpg', '131.jpg', '134.jpg', '136.jpg', '139.jpg', '141.jpg', '143.jpg', '146.jpg', '148.jpg']
1 ['103.jpg', '104.jpg', '120.jpg', '128.jpg', '133.jpg', '138.jpg', '149.jpg']
2 ['108.jpg', '110.jpg', '111.jpg', '112.jpg', '124.jpg', '125.jpg', '135.jpg', '137.jpg', '147.jpg']
3 ['118.jpg', '130.jpg']
4 ['123.jpg', '127.jpg', '144.jpg']
5 ['132.jpg', '140.jpg']
6 ['113.jpg']
7 []
8 ['122.jpg']
9 []
10 ['129.jpg']
11 ['109.jpg', '142.jpg', '145.jpg']

 game
0 ['131.jpg', '134.jpg', '139.jpg']
1 ['133.jpg', '138.jpg']
2 ['124.jpg']
3 ['130.jpg']
4 ['127.jpg']
5 ['132.jpg', '140.jpg']
6 []
7 []
8 []
9 []
10 []
11 []

 art
0 ['100.jpg', '101.jpg', '102.jpg', '105.jpg', '106.jpg', '107.jpg']
1 ['103.jpg', '104.jpg']
2 ['111.jpg', '112.jpg']
3 []
4 []
5 []
6 []
7 []
8 []
9 []
10 []
11 []
0 9


100%|██████████| 1141/1141 [01:31<00:00, 12.41it/s]


(1141, 4096)
每个类总计： [170, 201, 169, 121, 88, 58, 59, 31, 37, 33, 27, 147] 

0 ['101.jpg', '102.jpg', '103.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg', '114.jpg', '115.jpg', '116.jpg', '117.jpg', '119.jpg', '120.jpg', '121.jpg', '126.jpg', '128.jpg', '131.jpg', '133.jpg', '134.jpg', '136.jpg', '139.jpg', '141.jpg', '143.jpg', '146.jpg', '148.jpg']
1 ['110.jpg', '111.jpg', '125.jpg', '135.jpg', '138.jpg', '149.jpg']
2 ['108.jpg', '112.jpg', '123.jpg', '124.jpg', '130.jpg', '137.jpg', '147.jpg']
3 ['118.jpg', '127.jpg', '144.jpg']
4 ['113.jpg', '132.jpg', '140.jpg']
5 []
6 ['122.jpg', '129.jpg']
7 []
8 []
9 ['109.jpg']
10 []
11 ['142.jpg', '145.jpg']

 game
0 ['131.jpg', '133.jpg', '134.jpg', '139.jpg']
1 ['138.jpg']
2 ['124.jpg', '130.jpg']
3 ['127.jpg']
4 ['132.jpg', '140.jpg']
5 []
6 []
7 []
8 []
9 []
10 []
11 []

 art
0 ['100.jpg', '101.jpg', '102.jpg', '103.jpg', '104.jpg', '105.jpg', '106.jpg', '107.jpg']
1 ['111.jpg']
2 ['112.jpg']
3 []
4 []
5 []
6 []
7 []
8 []
9 []
10 []
11 [

In [71]:
os.makedirs('/cache/xiehaizhen/LSDIR_select', exist_ok=True)
for kk in range(1, 86):
    dir_name = str(kk).zfill(4) + '000'
    for r, d, f in os.walk(os.path.join('/cache/xiehaizhen/CurriculumClustering', dir_name)):
        if len(f) != 0:
            # print(r, d, f)
            for f_ in f:
                ori_path = os.path.join(r, f_)
                dst_path = os.path.join('/cache/xiehaizhen/LSDIR_select', r.split('/')[-2] + '_' + r.split('/')[-1] + '_' + f_)
                shutil.copy(ori_path, dst_path)

In [80]:
with open('LSDIR_select.txt', 'w') as file:
    for r, d, f in os.walk('/cache/xiehaizhen/CurriculumClustering'):
        if len(f) != 0:
            for f_ in f:
                ori_path = os.path.join(r, f_).replace('/cache/xiehaizhen/CurriculumClustering/', '')
                ori_path = ori_path.split('/')[0]+'/'+ori_path.split('/')[2]
                file.write(ori_path+'\n')